In [1]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import category_encoders as ce
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score, accuracy_score, balanced_accuracy_score
from scipy.stats import randint

In [2]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 100)

# Open, Read, and Save to data frame
X_train = '/Users/Dustin/Desktop/Lambda/Projects/2_Water Pump/train_features.csv'
X_test = '/Users/Dustin/Desktop/Lambda/Projects/2_Water Pump/test_features.csv'
y_train = '/Users/Dustin/Desktop/Lambda/Projects/2_Water Pump/train_labels.csv'

X_train = pd.read_csv(X_train)
X_test = pd.read_csv(X_test)
y_train = pd.read_csv(y_train)

# Will need to split X_train and y_train to training and validation
X_train.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [ ]:
X_test.head()

In [ ]:
X_train.isna().sum()

In [ ]:
X_train['construction_year'].value_counts()

## Feature Engineer

In [13]:

## X_train['long_and_lat'] = list(zip(X_train['longitude'], X_train['latitude']))
## X_test['long_and_lat'] = list(zip(X_test['longitude'], X_test['latitude']))

## X_train['is_construct'] = X_train['construction_year'] != 0
## X_test['is_construct'] = X_test['construction_year'] != 0

(X_train['amount_tsh'] / X_train['population']).value_counts()

0.000000       17108
inf              825
0.200000         342
50.000000        338
3.333333         320
20.000000        303
1.000000         284
0.250000         264
0.100000         256
1.666667         254
2.000000         243
10.000000        226
2.500000         216
0.333333         212
0.500000         211
0.166667         190
5.000000         185
500.000000       183
1000.000000      182
0.666667         149
6.666667         143
4.000000         134
0.133333         131
0.400000         128
12.500000        126
30.000000        122
200.000000       120
0.050000         112
2000.000000      109
4.166667         105
               ...  
0.012723           1
0.152905           1
12.585812          1
28.301887          1
6.637168           1
0.314465           1
0.038168           1
0.832178           1
0.047801           1
6.060606           1
14.354067          1
9.600000           1
64.285714          1
14.583333          1
0.034965           1
0.113020           1
0.019531     

In [ ]:
# Merge
#X_train = X_train.merge(y_train)

In [ ]:
# Remove cols and rows with NaN
# feature_remove = ['funder','installer','subvillage','public_meeting','scheme_name']
# observation_remove = ['scheme_management','permit']

# X_train = X_train.drop(columns=feature_remove).dropna(subset=observation_remove)
# X_test = X_test.drop(columns=feature_remove).dropna(subset=observation_remove)


In [ ]:
# Seperate X_train and y_train from X_train
#drop_cols= ['id']

# y_train = X_train.loc[:,['id','status_group']]
# X_train = X_train.drop(columns='status_group')
#X_test = X_test.drop(columns=drop_cols)

In [4]:
#y_train = y_train.drop(columns='id')

In [5]:
# look at shape
X_train.shape, y_train.shape, X_test.shape

((59400, 41), (59400, 1), (14358, 41))

In [6]:
# Split X_train to training and validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=42, test_size=.2)

In [ ]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape

In [7]:
# Encoder Categorical and fit_transform
encoder = ce.OrdinalEncoder()
X_train = encoder.fit_transform(X_train)


# Define parameters for the model
param_distributions = {
    'n_estimators': randint(50,300),
    'max_depth': randint(2,20)
}

# Define the model or input inside RSCV
model = RandomForestClassifier(n_jobs=-1, random_state=42)

# RSCV
search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_distributions,
    scoring='accuracy',
    n_iter=5,
    n_jobs=-1,
    cv=5,
    verbose=10,
    return_train_score=True,
    random_state=42
)

# Fit to training data
search.fit(X_train, y_train)

# Best Score? scoring=''
print('Training Accuracy Score:',search.best_score_)


Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   16.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   32.0s
[Parallel(n_jobs=-1)]: Done  13 out of  25 | elapsed:   32.8s remaining:   30.2s
[Parallel(n_jobs=-1)]: Done  16 out of  25 | elapsed:   33.5s remaining:   18.8s
[Parallel(n_jobs=-1)]: Done  19 out of  25 | elapsed:   47.7s remaining:   15.0s
[Parallel(n_jobs=-1)]: Done  22 out of  25 | elapsed:   49.9s remaining:    6.7s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:   52.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:   52.5s finished
C:\Users\Dustin\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:740: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


Training Accuracy Score: 0.8018939393939394


In [8]:
# Test Validation
best = search.best_estimator_
X_val = encoder.transform(X_val)
y_pred = best.predict(X_val)
print('Validation Set Accuracy Score:', accuracy_score(y_val, y_pred))

Validation Set Accuracy Score: 0.8076599326599326


In [9]:
import eli5
from eli5.sklearn import PermutationImportance

# Instantiate. prefit means cv already done
permuter = PermutationImportance(best, scoring='accuracy', cv='prefit', n_iter=3,
                                 random_state=42)
# Fit to test- Not sure if we use .values or without
permuter.fit(X_val, y_val)

# Show - Try without .tolist() if not then use it
feature_names = X_val.columns.tolist()
eli5.show_weights(permuter, top=None, feature_names=feature_names)

Weight,Feature
0.0387 ± 0.0029,quantity
0.0290 ± 0.0029,quantity_group
0.0182 ± 0.0002,construction_year
0.0137 ± 0.0016,longitude
0.0122 ± 0.0022,extraction_type_class
0.0119 ± 0.0030,waterpoint_type
0.0104 ± 0.0016,latitude
0.0096 ± 0.0006,amount_tsh
0.0063 ± 0.0006,lga
0.0062 ± 0.0015,gps_height


## RETEST

In [10]:
# Save features with number greater than 0
mask = permuter.feature_importances_ > 0
feature = X_train.columns[mask]
X_train = X_train[feature]

# Define parameters for the model
param_distributions = {
    'n_estimators': randint(50,300),
    'max_depth': randint(2,20)
}

# Define the model or input inside RSCV
model = RandomForestClassifier(n_jobs=-1, random_state=42)

# RSCV
search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_distributions,
    scoring='accuracy',
    n_iter=5,
    n_jobs=-1,
    cv=5,
    verbose=10,
    return_train_score=True,
    random_state=42
)

# Fit to training data
search.fit(X_train, y_train)

# Best Score? scoring=''
print('Training Accuracy Score:',search.best_score_)


Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   14.8s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   30.4s
[Parallel(n_jobs=-1)]: Done  13 out of  25 | elapsed:   31.0s remaining:   28.6s
[Parallel(n_jobs=-1)]: Done  16 out of  25 | elapsed:   31.5s remaining:   17.7s
[Parallel(n_jobs=-1)]: Done  19 out of  25 | elapsed:   46.3s remaining:   14.6s
[Parallel(n_jobs=-1)]: Done  22 out of  25 | elapsed:   48.2s remaining:    6.5s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:   50.6s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:   50.6s finished
C:\Users\Dustin\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:740: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


Training Accuracy Score: 0.8042718855218856


In [11]:
# Test Validation
best = search.best_estimator_
X_val = X_val[feature]
y_pred = best.predict(X_val)
print('Validation Set Accuracy Score:', accuracy_score(y_val, y_pred))

Validation Set Accuracy Score: 0.8082491582491582


In [ ]:
import matplotlib.pyplot as plt
# Length of columns
n = len(X_train.columns)

# Feature importances from best_estimator_
importances = pd.Series(best.feature_importances_, X_train.columns)

# Top feature sorted by descending value. Sliced from bottom to top
top_n = importances.sort_values()[-n:]

# Plot
plt.figure(figsize=(5,15))
top_n.plot.barh()
plt.show()

In [ ]:
# Test X_Test
best = search.best_estimator_
X_test = encoder.transform(X_test)
y_pred_test = best.predict(X_test)

In [ ]:
### SUBMISSION ###

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission['status_group'] = y_pred_test
submission.to_csv('First-submission.csv', index=False)

In [ ]:
search.best_estimator_

In [ ]:
y_pred_test.shape

In [ ]:
X_train.head()

In [ ]:
X_test.shape

In [ ]:
y_train['status_group'].value_counts(normalize=True)

In [ ]:
pd.Series(y_pred_test).value_counts(normalize=True)